# Let's build a language model
This lecture is inspired by and uses some code from Andrej Karpathy's excellent lecture series on NLP.
The original Transformers lecture can be watched on [Youtube](https://www.youtube.com/watch?v=kCc8FmEb1nY). I encourage you to do so: having two people explain the same usually benefits understanding a lot.

## Further reading
1. To broaden your understanding even further, you can also study an alternative implementation doing something similar, noting the differences. For example [GPT in 60 lines of code](https://jaykmody.com/blog/gpt-from-scratch/).
2. The original paper introducing the Transformer architecture: [Attention is all you need](https://arxiv.org/abs/1706.03762).
   


In [ ]:
# !pip install torch

In [ ]:
!wget -O DATA/shakespeare.txt https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt 
!cat DATA/shakespeare.txt | tr ' ' '\n' | sort | uniq -c | sort -nr | head -n50

In [ ]:
# Read all text.
with open('DATA/shakespeare.txt', 'r', encoding='utf-8') as f:
    text = f.read()
print(text[0:200])

In [ ]:
print(len(text))

## Build a simple next-character-prediction model

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

device = 'cuda' if torch.cuda.is_available() else 'cpu'

### Tokenizer
The tokenizer is super simple here: it's all characters that occur in the text.

In [ ]:
sorted(list(set(text)))

In [ ]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a character, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a character

In [ ]:
print(stoi)

In [ ]:
# print(text[0:100])
print([encode(c) for c in "Speak, speak!"])

### Train test split

In [ ]:
# We want to predict the next character

random_idx=15009

# example 1.1:
print(f"We'll use input '{text[random_idx:random_idx+1]}' to predict the next character '{text[random_idx+1:random_idx+2]}'.")


In [ ]:
for j in range(13):
    print(f"We'll use input '{text[random_idx:random_idx+j+1]}' to predict the next character '{text[random_idx+j+1:random_idx+j+2]}'.")

So one example of 13 characters is actually 13 sub examples!

We start with zero-length sequences because we want to make the model robust to starting from scratch.

Let's implement this in a structural way so that this whole data generation thing happens automatically. We'll also add some batches because we want to make sure the GPU's are flooded.

In [ ]:
# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [ ]:

block_size=13 # sometimes called context length, sequence length, ... also sometimes called the TIME dimension
batch_size=8 # independent sequences

random_idx = 15014
for j in range(13):
    print(f"We'll use input '{train_data[random_idx:random_idx+j+1]}' to predict the next character '{train_data[random_idx+j+1:random_idx+j+2]}'.")

In [ ]:
torch.randint(len(data) - block_size, (batch_size,)) # note the random sampling from the original data

In [ ]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,)) # note the random sampling from the original data
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

Let's try it out and see if it works

In [ ]:
x,y = get_batch("train")
for idx in range(3):
    xi=x[idx]
    yi=y[idx]
    print("----")
    print(f"x vector: ", xi)
    print("x text: ", "".join([itos[int(i)] for i in xi]))
    print(f"y vector: ", xi)
    print("y text: ", "".join([itos[int(i)] for i in yi]))

We are randomly sampling snippets of text of `block_size` long. Then, for each of those snippets we are creating multiple examples: for every character we want to predict the next character.

In [ ]:
print(x)
print(y)
for i in range(13):
    print(f"## Sub-example {i}")
    # print(x.shape) # so total nr of examples is 8*13
    print("x:", x[:,:i+1])
    # print(y.shape)
    print("y:", y[:, i])

## Model

In [ ]:

# super simple bigram model
class BigramLanguageModel(nn.Module):
    """Bi-gram model: predict characters based on previous character only."""
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (Batch, Time, Channel)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C) # get matrices in right format for F.cross_entropy
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
        
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        total_sequence = idx
        idx_next = idx
        for _ in range(max_new_tokens):
            # get the predictions
            logits, _ = self(idx_next)
            # apply softmax to get probabilities
            probs = F.softmax(logits[0], dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            
            total_sequence = torch.cat((total_sequence, idx_next), dim=1) # (B, T+1)    
        return total_sequence

model = BigramLanguageModel(vocab_size)
m = model.to(device)


In [ ]:
logits, loss = m(x, y)
logits
# logits.shape # One logit for every character in vocab, for every B*T

#### What does a random model do?

In [ ]:
# generate from random model
context = torch.zeros((1, 1), dtype=torch.long, device=device) #start with character 0 which is newline
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))

### Add a simple way to evaluate

In [ ]:
# Add some way to evaluate
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

### Training loop

In [ ]:

# create a PyTorch optimizer
learning_rate = 1e-2
max_iters = 10000
eval_iters=500
eval_interval=max_iters//5
def run_training_loop():
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
    for iter in range(max_iters):

        # every once in a while evaluate the loss on train and val sets
        if iter % eval_interval == 0:
            losses = estimate_loss()
            print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

        # sample a batch of data
        xb, yb = get_batch('train')

        # evaluate the loss
        logits, loss = model(xb, yb)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()
run_training_loop()


### Predictions

In [ ]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))

This is not really recognizable text yet, but you can tell that the model is doing something: it tries to make things that have a similar number of characters as words, sentences, punctuation, ...
From time to time, some actual common words like "The", "And" might already appear.

# Adding the first block of Attention

In [ ]:

n_embd = 32
head_size = n_embd // 1

dropout=0.2
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)


        
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # Self-attention takes a sequence, adds values of other tokens in the same sequence and outputs a new sequence of dim hs.
        # 
        # input of size (batch, time-step = sequence length = context length = block_size, channels = n_embed)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,head_size)
        q = self.query(x) # (B,T,head_size)
        # compute attention scores ("affinities")

        wei = q @ k.transpose(-2,-1)  # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        
        wei = wei * k.shape[-1]**-0.5 # keep everything in the same scale.
        
        # wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T) - make sure future tokens aren't accessible, only for decoder blocks. For encoder blocks: skip!
        
        wei = F.softmax(wei, dim=-1) # (B, T, T) - scale everything nicely to sum per row =1
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs) is the output of the head
        return out

class OneHeadedModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd) # maps characters to internal state per character
        self.position_embedding_table = nn.Embedding(block_size, n_embd) # maps positions of characters to internal state per index
        
        self.head = Head(head_size)

        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
    
        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,n_embd)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,n_embd): 0,1,2,3,... fixed sequence
        x = tok_emb + pos_emb # (B,T,n_embd)
        x = self.head(x) # (B,T,n_embd)

        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        """ Slightly different since we now have history"""
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = OneHeadedModel()
m = model.to(device)

In [ ]:
# Same training loop as before

learning_rate = 1e-3
max_iters = 10000
eval_interval=max_iters//5

run_training_loop()

In [ ]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))

## Adding multiple heads, and finish the whole block

In [ ]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity. This is applied per token separately."""

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size) # communication
        self.ffwd = FeedFoward(n_embd) # computation
        self.ln1 = nn.LayerNorm(n_embd) # normalization keeping things nicely unit gaussian (at init) per token
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x)) # note the `x +`: that's the residual connection for combating vanishing gradients
        x = x + self.ffwd(self.ln2(x))
        return x

In [ ]:
n_layer=1
n_head=4
class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

        # better init
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = GPTLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')


In [ ]:
run_training_loop()

In [ ]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))

## Just scale up!


In [ ]:
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length (here: number of characters) for predictions?
learning_rate = 3e-4
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2

max_iters = 3000
eval_interval=max_iters//10

model = GPTLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

In [ ]:
if False: # Re-train?
    # Takes about 15 min on V100.
    run_training_loop()
    torch.save(m.state_dict(), 'shakespeare_gpt.pth')
# step 0: train loss 4.3823, val loss 4.3756
# step 500: train loss 1.7285, val loss 1.8738
# step 1000: train loss 1.4017, val loss 1.6253
# step 1500: train loss 1.2790, val loss 1.5399
# step 2000: train loss 1.1935, val loss 1.5008
# step 2500: train loss 1.1329, val loss 1.4952
# step 3000: train loss 1.0720, val loss 1.5008

else:
    model = GPTLanguageModel()  # Replace with your model's class
    model.load_state_dict(torch.load('shakespeare_gpt.pth', map_location=torch.device(device)))
    model.eval()  # Set the model to evaluation mode

In [ ]:
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))

## Embeddings
Now, we'll dive into the model specifics and see a little trick

In [ ]:
# Tokenize the words (convert each character to its token ID)
words= ["ROMEO", "JULIET", "ELIZABETH","queen", "love"]
tokenized_words = [[stoi[char] for char in word] for word in words]

# Convert to tensor and pad sequences for equal length
max_len = max(len(t) for t in tokenized_words)
padded_tokens = [t + [0] * (max_len - len(t)) for t in tokenized_words]
input_ids = torch.tensor(padded_tokens).to(device)

# Forward pass
outputs = m(input_ids)

# Aggregating character embeddings to get word embeddings
# Here, using simple averaging
words_emb = outputs[0].mean(dim=1)

In [ ]:
m.lm_head.

In [ ]:
cos = nn.CosineSimilarity(dim=0, eps=1e-6)
for i in range(len(words)):
    for j in range(i):
        sim = cos(words_emb[i], words_emb[j])
        print(f"{words[i]}-{words[j]}: {sim:.2f}")